Update descriptions with already saved ones in description file.

In [13]:
import yaml
try:
    with open("parameters.yml", "r") as infile:
        parameters = yaml.safe_load(infile)
except FileNotFoundError:
    paramters = {}

In [14]:
parameters

{'l0': {'default': 1.0,
  'description': 'Base length unit of the simulation [default: 1.0 au]',
  'choices': ''},
 'm0': {'default': 1.0,
  'description': 'Base mass unit of the simulation [default: 1.0 solMass]',
  'choices': ''},
 'NRAD': {'default': 64,
  'description': 'Number of radial cells.',
  'choices': ''},
 'NSEC': {'default': 64,
  'description': 'Number of azimuthal cells',
  'choices': ''},
 'QUANTITIESRADIUSLIMIT': {'default': 5,
  'description': 'Limit the calculation of disk quantities to inside this radius. TODO: enlarge radius',
  'choices': ''},
 'DiskRadiusMassFraction': {'default': 0.99,
  'description': 'Calculation of the disk radius: this fraction of the total mass is contained within the disk radius.',
  'choices': ''},
 'ExponentialCellSizeFactor': {'default': 1.4,
  'description': 'Cell size increase factor to build an exponential grid. Only relevant for exponential grid type.',
  'choices': ''},
 'RadialSpacing': {'default': 'ARITHMETIC',
  'description': 

In [15]:
defaultfile = "default_config.yml"

with open(defaultfile, "r") as infile:
    defaults = yaml.safe_load(infile)

# clean up types
for key, val in defaults.items():
    t = val["type"]
    if "basic_string" in t:
        t = "string"
    elif t == "d":
        t = "double"
    elif t == "i":
        t = "int"
    elif t == "b":
        t = "bool"
    elif t == "j":
        t = "unsigned int"
    val["type"] = t

    

In [16]:
defaults

{'l0': {'default': 'none', 'type': 'string'},
 'm0': {'default': 'none', 'type': 'string'},
 't0': {'default': 1.0, 'type': 'string'},
 'temp0': {'default': 1.0, 'type': 'string'},
 'Rmin': {'default': 'none', 'type': 'double', 'unitsupport': True},
 'Rmax': {'default': 'none', 'type': 'double', 'unitsupport': True},
 'Nrad': {'default': 'none', 'type': 'unsigned int'},
 'Naz': {'default': 'none', 'type': 'unsigned int'},
 'DiskRadiusMassFraction': {'default': 0.99, 'type': 'double'},
 'QuantitiesRadiusLimit': {'default': 5,
  'type': 'double',
  'unitsupport': True},
 'ExponentialCellSizeFactor': {'default': 1.41, 'type': 'double'},
 'RadialSpacing': {'default': 'none', 'type': 'string'},
 'DoWrite1DFiles': {'default': True, 'type': 'bool'},
 'WriteDensity': {'default': True, 'type': 'bool'},
 'WriteVelocity': {'default': True, 'type': 'bool'},
 'WriteEnergy': {'default': True, 'type': 'bool'},
 'WriteTemperature': {'default': False, 'type': 'bool'},
 'WriteSoundSpeed': {'default': Fa

In [19]:
# remove old keys
lkeys_to_remove = []
param_old_lkeys = {key.lower() : key for key in parameters}
default_lkeys = {key.lower() : key for key in defaults}
for key in parameters:
    lkey = key.lower()
    if not lkey in default_lkeys:
        lkeys_to_remove.append(key)
for key in lkeys_to_remove:
    key = param_old_lkeys[key.lower()]
    parameters[key]["deprecated"] = True

new_params = {}

for key in defaults:
    lkey = key.lower()
    # copy the old param dict to an entry with new capitalization
    if lkey in param_old_lkeys:
        oldkey = param_old_lkeys[lkey]
        new_params[key] = parameters[oldkey]


    desc = ""
    defvalue = defaults[key]["default"]
    # add new parameters
    if not key in new_params:
        new_params[key] = {
            "default" : defvalue,
            "description" : desc,
            "choices" : ""
        }

    # update types
    new_params[key]["type"] = defaults[key]["type"]
    # update unitsupport
    if "unitsupport" in defaults[key] and defaults[key]["unitsupport"] == "yes":
        new_params[key]["unitsupport"] = True
    else:
        new_params[key]["unitsupport"] = False
    # update default values
    if new_params[key]["default"] != defvalue:
        new_params[key]["default"] = defvalue

# copy left over deprecated parameters
for key, val in parameters.items():
    if "deprecated" in parameters[key]:
        new_params[key] = parameters[key]
        

In [22]:
deprecated_params = {}
for key, val in new_params.items():
    if "deprecated" in val:
        deprecated_params[key] = val
        print(key, val)

NSEC {'default': 64, 'description': 'Number of azimuthal cells', 'choices': '', 'deprecated': True}
DomegaDrZero {'default': False, 'description': 'Set dOmega/dr = 0 at outer boundary', 'choices': 'yes, no', 'deprecated': True}
zbufferSize {'default': 100, 'description': '', 'choices': '', 'deprecated': True}
zbufferMaxAngle {'default': 0.17453292519943295, 'description': '', 'choices': '', 'deprecated': True}
CoolingRadiativeLocal {'default': False, 'description': 'Enable local radiative cooling Q- = 2 * sigma_R * T_eff^4.', 'choices': 'yes, no', 'deprecated': True}
CoolingScurve {'default': False, 'description': 'Enable Scurve Cooling', 'choices': 'yes, no', 'deprecated': True}
discmass {'default': 0.01, 'description': 'In case of SetSigma0=yes, use this mass as diskmass.', 'choices': '', 'deprecated': True}
IntegratePlanets {'default': True, 'description': 'Evolve the Nbody system.', 'choices': 'yes, no', 'deprecated': True}
massoverflow {'default': False, 'description': 'Enable a c

In [7]:

with open("new_parameters.yml", "w") as outfile:
    yaml.dump(new_params, outfile)

In [23]:
with open("deprecated_parameters.yml", "w") as outfile:
    yaml.dump(deprecated_params, outfile)

In [9]:
for key, val in new_params.items():
    if "oldname" in val:
        print(key, val["oldname"])

ViscousAlpha AlphaViscosity
ConstantViscosity Viscosity


In [28]:
with open("deprecated_parameters.yml", "r") as infile:
    deprecated = yaml.safe_load(infile)

for key, val in deprecated.items():
    newname = val["newname"]
    if newname == "none":
        if "hint" in val:
            hint = val["hint"]
            print(f"{key}: have a look at {hint}")
        else:
            print(f"{key} removed without replacement")
    else:
        print(f"{key} : {val['newname']}")

ALPHAVISCOSITY : ViscousAlpha
CoolingRadiativeLocal : SurfaceCooling: thermal
CoolingScurve : SurfaceCooling: scurve
DT : MonitorTimestep
DomegaDrZero removed without replacement
IntegratePlanets removed without replacement
NSEC : Naz
NTOT : Nsnap
StellarRotation : InnerBoundaryVaziKeplerianFactor, InnerBoundaryVazi = keplerian
VISCOSITY : ConstantViscosity
VRadIn : InnerBoundaryVradKeplerianFactor, InnerBoundaryVrad = keplerian
discmass : DiskMass
massoverflow : RocheLobeOverflow
mofaveragingtime : ROFAveragingTime
mofgamma : ROFGamma
mofplanet : ROFPlanet
moframpingtime : ROFRampingTime
moftemperature : ROFTemperature
mofvalue : ROFValue
variableTransfer : ROFVariableTransfer
zbufferMaxAngle removed without replacement
zbufferSize removed without replacement
